In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable, grad

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy
import scipy.io as io
from pyDOE import lhs

from sklearn.metrics import mean_squared_error, mean_absolute_error

from pysr import pysr, best, best_callable

### For automatic PDE recovery ###
from gplearn.genetic import SymbolicRegressor
import sympy
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
from sympy.core import evaluate
from utils import *

In [2]:
data = io.loadmat('data/burgers_shock.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:, None]))
u_star = Exact.flatten()[:,None]

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

N = 2000
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

In [3]:
choice = 'pysr'

if choice == 'gplearn':
    ### Using a gplearn model ###
    program = pickle_load('./data/gp_symreg.pkl')._program
elif choice == 'pysr':
    ### Using a pysr model ###
    program = best(pickle_load('./saved_path_inverse_burger/equations_from_pysr.pkl'))

expr, vars = build_exp(program)
expr = expr.subs(Integer(-1), Float(-1.0, precision=53))
# expr = manipulate_expr(expr)
print(expr)

Loaded from ./saved_path_inverse_burger/equations_from_pysr.pkl
-0.6860516*x0*x1 + 0.0020575821*x2


In [4]:
constants = list(expr.atoms(Number))

# Replace all positive coeffs with Sympy symbols 
with evaluate(False):
    for i in range(len(constants)):
        var = Symbol('C'+str(i))
        expr = expr.subs(abs(constants[i]), var)
variables = list(expr.atoms(Symbol))

# Use the log form for the parameters that are less than 0.005
# Have to store the indices which require torch.exp
exp_indices = []
for i in range(len(constants)):
    constants[i] = abs(constants[i])
    if constants[i] < 0.005:
        constants[i] = log(constants[i])
        exp_indices.append(i)

pde_params = nn.Parameter(data=torch.tensor(constants, dtype=torch.float32))

In [5]:
### Sorting Symbols ###
values = [string2int(str(e)) for e in variables]
tmp = []
for i in range(len(values)):
    tmp.append((values[i], variables[i]))
tmp.sort()
variables = [e[1] for e in tmp]
del tmp

variables

[C0, C1, x0, x1, x2]

In [6]:
fn = lambdify(tuple(variables), expr)

print('-- loss_fn --')
print(expr)

print('-- pde_params --')
print(pde_params)

-- loss_fn --
C0*x2 + (-C1)*x0*x1
-- pde_params --
Parameter containing:
tensor([-6.1862,  0.6861], requires_grad=True)


In [7]:
class Network(nn.Module):
    def __init__(self, model, pde_parameters, loss_fn, exp_indices):
        super(Network, self).__init__()
        self.model = model
        self.model.apply(self.xavier_init)
        self.callable_loss_fn = loss_fn
        self.pde_parameters = pde_parameters
        self.exp_indices = exp_indices
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, x, t):
        return self.model(torch.cat([x, t], dim=1))
    
    def loss(self, x, t, y_input, update_network_params=True, update_pde_params=True):
        total_loss = 0.0
        uf = self.forward(x, t)
        if update_network_params:
            mse_loss = self.mean_squared(uf - y_input)
            total_loss = total_loss + mse_loss
        
        if update_pde_params:
            tmp_params = torch.zeros(self.pde_parameters.shape)
            for idx in range(self.pde_parameters.shape[0]):
                if idx in self.exp_indices: 
                    tmp_params[idx] = torch.exp(self.pde_parameters[idx])
                else: 
                    tmp_params[idx] = self.pde_parameters[idx]
                
            # PDE Loss calculation
            u_t = self.gradients(uf, t)[0]
            u_x = self.gradients(uf, x)[0]
            u_xx = self.gradients(u_x, x)[0]
            
            l_eq = self.mean_squared(u_t-self.callable_loss_fn(tmp_params[0], tmp_params[1], uf, u_x, u_xx))

            total_loss = total_loss + l_eq
            
        return total_loss
    
    def get_gradients_dict(self, x, t):
        self.eval()
        
        uf = self.forward(x, t)
        ß
        ### PDE Loss calculation ###
        # first-order derivatives
        u_t = self.gradients(uf, t)[0]
        u_x = self.gradients(uf, x)[0]
        # Homo second-order derivatives
        u_tt = self.gradients(u_t,t)[0]
        u_xx = self.gradients(u_x, x)[0]
        # Hetero second-order derivatives
        u_xt = self.gradients(u_t, x)[0]
        u_tx = self.gradients(u_x, t)[0]
        
        return {'uf':uf, 'u_x':u_x, 'u_xx':u_xx}, u_t
    
    def mean_squared(self, inn_tensor):
        return (inn_tensor**2).mean()
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))

In [8]:
pretrained = False
hidden_nodes = 50

model = nn.Sequential(nn.Linear(2, hidden_nodes), 
                        nn.Tanh(), 
                        nn.Linear(hidden_nodes, hidden_nodes),
                        nn.Tanh(), 
                        nn.Linear(hidden_nodes, hidden_nodes),
                        nn.Tanh(), 
                        nn.Linear(hidden_nodes, hidden_nodes),
                        nn.Tanh(),
                        nn.Linear(hidden_nodes, 1))

network = Network(model=model, pde_parameters=pde_params, loss_fn=fn, exp_indices=exp_indices)
if pretrained: network.load_state_dict(torch.load("./saved_path_inverse_burger/nn_without_physical_reg.pth"), strict=False)

In [9]:
X_u_train = torch.tensor(X_u_train).float().requires_grad_(True)
u_train = torch.tensor(u_train).float().requires_grad_(True)

X_star = torch.tensor(X_star).float().requires_grad_(True)
u_star = torch.tensor(u_star).float().requires_grad_(True)

In [10]:
# optimizer = torch.optim.Adam(network.parameters(), lr=5e-4)  # metaopt also has .parameters()
optimizer = torch.optim.LBFGS(network.parameters(), lr=5e-2, max_iter=50, max_eval=50, line_search_fn='strong_wolfe')
epochs = 500
# weights_path = './saved_path_inverse_burger/frozen_pinn.pth'
# weights_path = './saved_path_inverse_burger/nn_with_physical_reg_from_symreg.pth'

In [11]:
network.train(); best_train_loss = 1e6
for i in range(epochs):
    ### Add the closure function to calculate the gradient. For LBFGS.
    def closure():
        if torch.is_grad_enabled():
            optimizer.zero_grad()
        l = network.loss(X_u_train[:, 0:1], X_u_train[:, 1:2], u_train, update_network_params=True, update_pde_params=True)
        if l.requires_grad:
            l.backward()
        return l

    optimizer.step(closure)
    l = closure()

#     if i > 400 and float(l.item()) < best_train_loss:
#         torch.save(network.state_dict(), weights_path)
#         best_train_loss = float(l.item())

    if (i % 100) == 0:
        print("Epoch {}: ".format(i), l.item())

Epoch 0:  0.049615528434515
Epoch 100:  4.703639933723025e-05
Epoch 200:  3.808536712313071e-05
Epoch 300:  3.808536712313071e-05
Epoch 400:  3.808536712313071e-05


In [12]:
est_lambdas = [e.detach().item() for e in network.pde_parameters]
for idx in exp_indices:
    est_lambdas[idx] = np.exp(est_lambdas[idx])
est_lambdas

[0.0031686916882638083, 0.991890013217926]

In [13]:
### Loading the best weights ###
# network.load_state_dict(torch.load(weights_path))

In [14]:
network.eval()

Network(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Tanh()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Tanh()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Tanh()
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Tanh()
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [15]:
### pls chk this code before actual evaluation ###
nu = 0.01 / np.pi
grounds = [nu, 1.0]

error_lambda_1 = np.abs(est_lambdas[0] - grounds[0]) / nu * 100
error_lambda_2 = np.abs(est_lambdas[1] - grounds[1])*100

error_lambda_1, error_lambda_2 # (0.3306746482849121, 1.30792548173677)

(0.4526147065938181, 0.8109986782073975)

### Symbolic regression

In [ ]:
grads_dict, target = network.get_gradients_dict(X_u_train[:, 0:1], X_u_train[:, 1:2])
index2features = grads_dict.keys()
print(index2features)

G = torch.cat(list(grads_dict.values()), dim=1).detach().numpy()
target = torch.squeeze(target).detach().numpy()

In [ ]:
equations = pysr(G, target, niterations=100, binary_operators=["plus", "sub", "mult"], unary_operators=[], batching=True)

In [ ]:
print(best(equations))
fn = best_callable(equations)

In [ ]:
uf = grads_dict['uf']
u_x = grads_dict['u_x']
u_xx = grads_dict['u_xx']

In [ ]:
# Exreacted equation (for further fine-tuning)
# u_t + 0.6860763*uf*u_x - 0.0020577204*u_xx